In [ ]:
import pandas as pd 
import csv

In [ ]:
path="/kaggle/input/final-dataset/resampled data.csv"
re_df=pd.read_csv(path)
re_df.head()

In [ ]:
re_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
re_df.info()

In [ ]:
weights = {
    "toeflScore": 0.96609756,
    "GRE_AWA": 0.72707317,
    "GRE_QUANT": 0.84658537,
    "GRE_VERBAL": 0.99,
    "GPA": 0.70317073,
    "Course Name Encoded": 0.72707317,
}

In [ ]:
cols=weights.keys()
w=weights.values()
cols,w

In [ ]:
re_df['Score']=re_df['toeflScore']*weights['toeflScore']+re_df['GRE_AWA']*weights['GRE_AWA']+re_df['GRE_QUANT']*weights['GRE_QUANT']+re_df['GRE_VERBAL']*weights['GRE_VERBAL']+re_df['GPA']*weights['GPA']
re_df.head()

In [ ]:
min_score = re_df['Score'].min()

# Normalize the Score column
re_df['Normalized Score'] = re_df['Score'] - min_score

### Changes to the input
1. GPA to 4
2. Scores-130
3. Normalize using robust scalar

### Collaborative- Memory Based

In [ ]:
X=re_df[['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded']]
y=re_df['University Name Encoded']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train)

In [ ]:
X

#  Hybrid Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(5,), kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(1024, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(2056, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(2056, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(1024, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal()),
    BatchNormalization(),
    Dense(1)  # Output layer without activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Print model summary
model.summary()


In [ ]:
# Assuming you have prepared your input data (X_train) and target data (y_train)

# Train the model
history=model.fit(X_train, y_train, epochs=50, batch_size=1024, validation_split=0.2)


In [ ]:
import matplotlib.pyplot as plt


# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
predictions = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming model.predict(X_test) returns the predicted values
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")


In [ ]:
predictions

In [ ]:
import numpy as np
import pandas as pd

# Define batch size for processing
batch_size = 10000

# Calculate number of batches
num_batches = len(X_train) // batch_size + 1

# Create an empty list to store predicted ratings
predicted_ratings_list = []

# Iterate through batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_train))
    batch_data = X_train.iloc[start_idx:end_idx]
    
    # Prepare input data as a 2D array
    input_data = np.array(batch_data[['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded']])
    
    # Predict ratings using the model
    predicted_ratings = model.predict(input_data)
    
    # Extend the list with batch predicted ratings
    predicted_ratings_list.extend(predicted_ratings)

# Create DataFrame from the list of predicted ratings
predicted_ratings_df = pd.DataFrame(predicted_ratings_list, columns=['predicted_rating'])
predicted_ratings_df[['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded']] = X_train[['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded']]


In [ ]:
predicted_ratings_df.head()

In [ ]:
X_train

In [ ]:
predicted_ratings_df.info()

In [ ]:
y_train

In [ ]:
predicted_ratings_df=pd.read_csv("/kaggle/working/Ratings.csv")

In [ ]:
y_train_list=y_train.to_list()

In [ ]:
predicted_ratings_df['University Name']=y_train_list

In [ ]:
predicted_ratings_df.isna().sum()

In [ ]:
predicted_ratings_df.to_csv('Ratings2.csv')

In [ ]:
predicted_ratings_df.drop(['GRE_AWA','GRE_QUANT','GRE_VERBAL',"GPA","Course Name Encoded"],axis=1,inplace=True)
predicted_ratings_df.head()

In [ ]:
predictions

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Assuming X_train contains the scores and y_train contains the corresponding university names or identifiers

k = 50  # You can adjust this value based on your needs

# Fit KNN model
X_train_reshaped = predicted_ratings_df['predicted_rating'].values.reshape(-1, 1)

knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(X_train_reshaped, predicted_ratings_df['University Name'])

# for pred in predictions:
#     # Example: Find nearest score to a given score
#     given_score = pred[0]  # The score you want to find the nearest neighbor for
#     given_score_vector = [[given_score]]  # KNN expects input as a 2D array
#     nearest_score = knn_model.predict(given_score_vector)

#     print("Nearest score to", given_score, "is:", nearest_score)


In [ ]:
preds = []
i=0
# predictions=model.predict(X_test)
for pred in predictions:
    given_score = pred[0]  # The score you want to find the nearest neighbors for
    given_score_vector = [[given_score]]  # KNN expects input as a 2D array
    
    # Use kneighbors to find the top 20 nearest scores
    distances, indices = knn_model.kneighbors(given_score_vector, n_neighbors=40)
#     print(distances, indices)
    # Retrieve the corresponding universities
    nearest_universities = predicted_ratings_df.iloc[indices[0]]['University Name'].values
#     unique_universities = list(set(nearest_universities))[:20]  # Get unique universities and limit to 20
    
    # Append the nearest universities to preds
    preds.append(list(set(nearest_universities))[:40])


In [ ]:
##training acc
preds_training = []
predictions=model.predict(X_train)
i=0
for pred in predictions:
    given_score = pred[0]  # The score you want to find the nearest neighbors for
    given_score_vector = [[given_score]]  # KNN expects input as a 2D array
    
    # Use kneighbors to find the top 20 nearest scores
    distances, indices = knn_model.kneighbors(given_score_vector, n_neighbors=40)
#     print(distances, indices)
    # Retrieve the corresponding universities
    nearest_universities = predicted_ratings_df.iloc[indices[0]]['University Name'].values
#     unique_universities = list(set(nearest_universities))[:20]  # Get unique universities and limit to 20
    
    # Append the nearest universities to preds
    preds_training.append(list(set(nearest_universities))[:40])


In [ ]:
y_test_list=y_test.to_list()

In [ ]:
s=0
for i in range(len(preds_training)):
    if y_train_list[i] in preds_training[i]:
        s+=1

In [ ]:
s/len(preds_training)

In [ ]:
s=0
for i in range(len(preds)):
    if y_test_list[i] in preds[i]:
        s+=1

In [ ]:
s/len(preds)

In [ ]:
model.save('final_model.h5')

# Code for testing data starts

In [ ]:
### Recommendation System

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Assuming your model is saved in a file named 'my_model.h5'
model_path = "/kaggle/input/rec/keras/new/1/nn_for_uni_rec_wt_toefl.h5"

# Load the model
model = load_model(model_path)

In [ ]:
import pandas as pd

path="/kaggle/input/unresampled-dataset-prefinal/final dataset using knn.csv"
df=pd.read_csv(path)
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
numerical_columns = ['toeflScore', 'GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA','min_gpa','gre_total']

for col in numerical_columns:
  if col=='min_gpa' or col=='GPA' or col=='GRE_AWA':
    df[col]=df[col].astype(float).round(2)
  else:
    df[col]=df[col].astype(int)

df.head()

In [ ]:
df = df.groupby('Course Name').filter(lambda x: len(x) >= 50)

In [ ]:
df.info()

In [ ]:
profile={
    'toeflScore':92,
    'GRE_AWA':4,
    'GRE_QUANT':165,
    'GRE_VERBAL':155,
    'GPA': 9,
    'Publications':1,
    'Work experience':1,
    'Research Experience':0,
    'Course Name Encoded':1,
    'Degree': 1,
    'Budget': 80000
}
profile_df = pd.DataFrame([profile])

#add budget******************************************************

In [ ]:
profile_df

In [ ]:
# Course name to encoded format

In [ ]:
df.head()

In [ ]:
#Check if in range
#to be done on frontend

In [ ]:
# GRE
profile_df['GRE_QUANT'] = profile_df['GRE_QUANT'] - 130
profile_df['GRE_VERBAL'] = profile_df['GRE_VERBAL'] - 130

In [ ]:
#Convert GPA to scale of 4
import pandas as pd

# GPA conversion function
def convert_gpa(gpa, scale):
    if scale == 10:
        return gpa / 2.5
    elif scale ==5:
        return (gpa/5)*4
    elif scale == 20:
        return gpa / 5.0
    elif scale == 100:
        return gpa / 25.0
    else:
        return gpa  

# Convert GPA column to a 4.0 scale
profile_df['GPA'] = profile_df.apply(lambda row: convert_gpa(row['GPA'], 10), axis=1)  # Assuming GPA is on a 10-point scale

# Display the updated DataFrame
print(profile_df)


In [ ]:
df = pd.concat([df, profile_df], ignore_index=True)


In [ ]:
df.drop(['min_gpa','gre_total','min_gre'],axis=1,inplace=True)

In [ ]:
### Preprocessing the profile
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
columns=['GPA','GRE_AWA','GRE_QUANT','GRE_VERBAL','toeflScore','Publications','Research Experience','Work experience']
df[columns] = scaler.fit_transform(df[columns])

# Display the DataFrame
print(df)

In [ ]:
ip_df = df.iloc[[-1]]
ip_df

In [ ]:
input_df=ip_df[['GRE_AWA','GRE_QUANT','GRE_VERBAL','GPA','Course Name Encoded']]

In [ ]:
logits=model.predict(input_df)
# Apply softmax to get probabilities
probabilities = tf.nn.softmax(logits)

# Convert TensorFlow tensor to numpy array
probabilities = probabilities.numpy()

# Display predicted probabilities
print(probabilities)

In [ ]:
len(probabilities)

In [ ]:
import numpy as np


# Extract probabilities and corresponding class labels
probabilities = logits.ravel()  # Flatten the array
class_labels = np.arange(len(probabilities))  # Assuming class indices start from 0

# Get indices of top 10 predicted classes
top_10_indices = np.argsort(probabilities)[::-1][:40]

# Get corresponding probabilities and class labels for top 10
top_10_probabilities = probabilities[top_10_indices]
top_10_class_labels = class_labels[top_10_indices]

# Display top 10 predicted classes along with their probabilities
predictions=[]
for i in range(20):
    predictions.append([top_10_class_labels[i], top_10_probabilities[i]])
    filtered_df = re_df[re_df['University Name Encoded'] == top_10_class_labels[i]]
#     print(filtered_df.head())
    max_score=filtered_df['toeflScore'].max()*weights['toeflScore']+filtered_df['GRE_AWA'].max()*weights['GRE_AWA']+filtered_df['GRE_QUANT'].max()*weights['GRE_QUANT']+filtered_df['GRE_VERBAL'].max()*weights['GRE_VERBAL']+filtered_df['GPA'].max()*weights['GPA']
#     print(max_score)
#     max_score=filtered_df['Normalized Score'].max()
    print(max_score)
    score=ip_df['GRE_AWA']*weights['GRE_AWA']+ip_df['GRE_QUANT']*weights['GRE_QUANT']+ip_df['GRE_VERBAL']*weights['GRE_VERBAL']+ip_df['GPA']*weights['GPA']
    admit_percent=((score-min_score)/(max_score-min_score))*100
    print("Class:", top_10_class_labels[i], " Probability:", top_10_probabilities[i], " Admit Percent:", admit_percent)

In [ ]:
#converting class names to university names

import pandas as pd

# Specify the path to your text file
file_path = "/kaggle/input/university-and-courses-encoded/university_rankings.txt"

universities_dict = {}

# Open the text file
with open(file_path, 'r') as file:
    # Read each line from the file
    for line in file:
        # Split the line into key and value
        key, value = line.strip().split(': ')
        # Convert key to integer
        value = int(value)
        # Store the key-value pair in the dictionary
        universities_dict[value] = key

# Print the dictionary
# print(universities_dict)


In [ ]:
recommended_list=[]
for pred in predictions:
    recommended_list.append([universities_dict[pred[0]],pred[1]])
    print(pred[0],universities_dict[pred[0]],pred[1])

In [ ]:
recommended_list[:5]

In [ ]:
# check predicted data
checks_df=pd.read_csv('/kaggle/input/merged-limits/Merged_Limits.csv')
checks_df.head()

In [ ]:
checks_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
checks_df['Avg Living Expenses in dollar_x'] = checks_df['Avg Living Expenses in dollar_x'].str.replace(',', '').str.replace('$', '').astype(int)


In [ ]:
cols=checks_df.columns
cols

In [ ]:
for rec in recommended_list:
    result = checks_df[checks_df['University'] == rec[0]]
    rank = checks_df[checks_df['University'] == rec[0]]
    if ((profile['toeflScore'] < result['toeflScore']).any() or
        ((profile['GRE_QUANT'] + profile['GRE_VERBAL']) < result['min_gre']).any() or
        (profile['GPA'] < result['min_gpa']).any() or
        (profile['Budget'] < result['Avg Living Expenses in dollar_x']).any()):
        pass
    else:
        rec.append(rank['Rank'].tolist()[0])


In [ ]:
print(recommended_list)

In [ ]:
len(recommended_list)

### Feedback loop

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Assuming your model is saved in a file named 'my_model.h5'
model_path = "/kaggle/input/rec/keras/new/1/nn_for_uni_rec_wt_toefl.h5"

# Load the model
model = load_model(model_path)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
import pandas as pd
profile=[165,155,4,9,1,103]
profile_df = pd.DataFrame([profile], columns=['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded','University Name Encoded'])
profile_df

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming you have imported and defined your Keras model as 'model' already

# Start the feedback loop
while True:
    # Gather feedback on model performance
    user_feedback = input("Was the model prediction correct? (yes/no): ")
    
    if user_feedback.lower() == 'no':
        # Gather more information about the misclassified sample
        misclassified_sample_text_np = profile_df.values
        correct_label = 110
        
        # Gather correct label from the user
        user_correct_label = int(input("Enter the university: "))
        
        # Load model weights
        model.load_weights('/kaggle/input/feature-weights/keras/universityrecfeatures/1/best_model.keras')

        X_train = np.concatenate([X_train, profile_df[['GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA', 'Course Name Encoded']]])
        y_train = np.concatenate([y_train, profile_df['University Name Encoded']])        # Retrain the model with the corrected label
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=30)
        
        # Evaluate the updated model
        updated_accuracy = model.evaluate(X_test, y_test)
        print("Updated Model Accuracy:", updated_accuracy)
        
    elif user_feedback.lower() == 'yes':
        print("Great! Model performance is satisfactory.")
        break
    
    else:
        print("Invalid input. Please enter 'yes' or 'no'.")

# Save the updated model
model.save('updated_model.h5')


In [ ]:
len(X_train)

In [ ]:
# 2252569

In [ ]:
import json
with open("university_courses.json", "w") as json_file:
    json.dump(university_courses_dict, json_file, indent=4)

print("Data saved to university_courses.json")